<img src="resources/titanic_sinking.gif" alt="Titanic sinking gif" style="margin: 10px auto 20px; border-radius: 15px;" width="100%"/>

<a id="project-overview"></a>
_**Potonuće Titanika** jedno je od najozloglašenijih brodoloma u istoriji._

_15. aprila 1912. godine, tokom njegovog prvog putovanja, široko smatrani „nepotopivi“ RMS Titanic potonuo je nakon sudara sa santom leda u severnom Atlantskom okeanu. Nažalost, nije bilo dovoljno čamaca za spasavanje za sve na brodu, što je rezultovalo smrću 1502 od 2224 putnika i posade._

_Iako je u preživljavanju učestvovao neki element sreće, čini se da su neke grupe ljudi imale veću verovatnoću preživljavanja u odnosu na druge._

Ovaj projekat predstavlja prediktivni model koji odgovara na pitanje: _„za koje vrste ljudi je veća verovatnoća da će preživeti?“_ korišćenjem dostupnih podataka o putnicima (tj. ime, starost, pol, socijalno-ekonomska klasa, itd.).

<hr>

<a id="table-of-contents"></a>
### Pregled sadržaja:
* [Pregled projekta](#project-overview)
* [Učitavanje skupa podataka](#loading-datasets)
* [Provera relevantnosti kolona za model](#column-relevance)
* [Model - Random Forest](#model-randfor)
* [Model - KNN](#model-knn)

<hr>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### Učitavanje skupa podataka <a id="loading-datasets"></a>
<a href="#table-of-contents"> Povratak na pregled sadržaja </a>

<div style="display: inline-block;">

| Variable | Definition                               | Key                                            |
|----------|------------------------------------------|------------------------------------------------|
| Survived | Yes or No                                | 1 = Survived, 0 = Died                                         |
| Pclass   | A proxy of socio-economic status (SES)   | 1 = Upper,  2 = Middle, 3 = Lower              |
| Name     | Passenger name                           | e.g. Allen, Mr. William Henry                  |
| Sex      | Passenger sex                            | male or female                                 |
| Age      | Passenger age                            | integer                                        |
| SibSp    | # of siblings/spouses aboard the Titanic | integer                                        |
| Parch    | # of parents/children aboard the Titanic | integer                                        |
| Ticket   | Ticket number                            | e.g. A/5 21171                                 |
| Fare     | Passenger fare                           | float                                          |
| Cabin    | Cabin number (str)                       | e.g. B42                                       |
| Embarked | Port of Embarkation                      | C = Cherbourg, Q = Queenstown, S = Southampton |
    
    
</div>

- **train.csv** sadrži detalje o podskupu putnika na brodu (tačnije 891 putnik - gde svaki putnik predstavlja zaseban red u tabeli).
- **test.csv** sadrži detalje o podskupu putnika na brodu (tačnije 418 putnika - gde svaki putnik predstavlja zaseban red u tabeli) ** **bez 'Survived' kolone**.

In [49]:
# ucitavanje skupova podataka
train_df = pd.read_csv("datasets/train.csv")
test_df = pd.read_csv("datasets/test.csv")

### Provera relevantnosti kolona za model<a id="column-relevance"></a>

Potrebno je proveriti koje od kolona uticu na rezultat prezivljavanja odredjenog putnika.

- Sex - _Izracunavanjem srednje vrednosti smo ustanovili da je **pol** putnika igrao bitnu ulogu u prezivljavanju_
- Pclass - _Izracunavanjem srednje vrednosti smo ustanovili da je **socijalno-ekonomska klasa** putnika igrala bitnu ulogu u prezivljavanju_

<a href="#table-of-contents"> Povratak na pregled sadržaja </a>

In [48]:
# Pol
men = np.mean(train_df.loc[train_df.Sex == 'male']["Survived"])
women = np.mean(train_df.loc[train_df.Sex == 'female']["Survived"])

print(f'Verovatnoca da ce muskarci preziveti iznosi {men * 100:.2f}%, a zene {women * 100:.2f}%\n')

# Pclass
upper = np.mean(train_df.loc[train_df.Pclass == 1]["Survived"])
middle = np.mean(train_df.loc[train_df.Pclass == 2]["Survived"])
lower = np.mean(train_df.loc[train_df.Pclass == 3]["Survived"])

print(f'Verovatnoca da ce visa klasa preziveti iznosi {upper * 100:.2f}%',
      f'srednja {middle * 100:.2f}%',
      f'i niza klasa {lower * 100:.2f}%\n')

Verovatnoca da ce muskarci preziveti iznosi 18.89%, a zene 74.20%

Verovatnoca da ce visa klasa preziveti iznosi 62.96% srednja 47.28% i niza klasa 24.24%



### Model - Random Forest <a id="model-randfor"></a>

<a href="#table-of-contents"> Povratak na pregled sadržaja </a>

In [45]:
columns = ["Sex", "Pclass", "SibSp", "Parch"]
train_x = pd.get_dummies(train_df[columns])
train_y = train_df["Survived"]

test_x = pd.get_dummies(test_df[columns])

random_forest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
random_forest.fit(train_x, train_y)
predictions = random_forest.predict(test_x)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})

output

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### Model - KNN <a id="model-knn"></a>

<a href="#table-of-contents"> Povratak na pregled sadržaja </a>

In [47]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_x, train_y)
predictions = knn.predict(test_x)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})

output

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
